In [19]:
def extract_item(request):
    """
    提取request的内容
    :param request:
    :return:
    """
    item = dict()
    item["url"] = request.url
    item["text"] = request.response.text or ""
    item["status_code"] = request.response.status_code or 0
    return item

def map(requests, stream=False, size=None, exception_handler=None, gtimeout=None):
    """Concurrently converts a list of Requests to Responses.

    :param requests: a collection of Request objects.
    :param stream: If True, the content will not be downloaded immediately.
    :param size: Specifies the number of requests to make at a time. If None, no throttling occurs.
    :param exception_handler: Callback function, called when exception occured. Params: Request, Exception
    :param gtimeout: Gevent joinall timeout in seconds. (Note: unrelated to requests timeout)
    """
    requests = list(requests)
    pool = Pool(size) if size else None
    jobs = [send(r, pool, stream=stream) for r in requests]
    gevent.joinall(jobs, timeout=gtimeout)
    ret = []
    for request in requests:

        if request.response is not None:
            ret.append(extract_item(request))
        elif exception_handler and hasattr(request, 'exception'):
            ret.append(exception_handler(request, request.exception))
        else:
            ret.append(None)

    yield ret

In [25]:
import grequests

urls = [
    'http://www.baidu.com',
    'http://www.qq.com',
    'http://www.163.com',
    'http://www.zhihu.com',
    'http://www.toutiao.com',
    'http://www.douban.com1'
]
rs = (grequests.get(u) for u in urls)
response_list = grequests.map(rs, size=100, gtimeout=100)
for response in (response_list):
    print(response.url+" "+str(response.status_code))

http://www.baidu.com/ 200
https://www.qq.com/ 200
http://www.163.com/ 403
https://www.zhihu.com/ 403
https://www.toutiao.com/ 200


AttributeError: 'NoneType' object has no attribute 'url'